In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import datetime as dt

from sympy import symbols, Symbol, Eq, nsolve, solve, sqrt
from decimal import Decimal, getcontext

import multiprocessing as mp

from LP_Action_Liquidity_Parameter_Function import Get_Liquidity_Parameter, Get_Price

data_dir = '/Users/zhicong/Dropbox/DEXs'
pool_name = 'USDC-WETH 500 10'

# LP Action

In [2]:
lp_action = pd.read_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + ".csv"))
lp_action.columns = [x.upper() for x in lp_action.columns.tolist()]
lp_action.dropna(subset = ['AMOUNT0_USD','AMOUNT1_USD'], inplace = True)

# Final cleaning
lp_action['BLOCK_NUMBER'] = lp_action['BLOCK_NUMBER'].astype(int)
lp_action.sort_values(by = ['BLOCK_NUMBER'], ascending = False, inplace = True)
lp_action['AMOUNT0_ADJUSTED'] = lp_action['AMOUNT0_ADJUSTED'].astype(float)
lp_action['AMOUNT1_ADJUSTED'] = lp_action['AMOUNT1_ADJUSTED'].astype(float)
lp_action['BLOCK_TIMESTAMP'] = pd.to_datetime([x[:-5] for x in lp_action['BLOCK_TIMESTAMP'].astype(str)])
lp_action = lp_action[(lp_action['AMOUNT0_ADJUSTED'] != 0) | (lp_action['AMOUNT1_ADJUSTED'] != 0)]
lp_action['NF_TOKEN_ID'] = lp_action['NF_TOKEN_ID'].apply(str).str.replace('.0','', regex = False)

# Changing signs of addition and extraction
lp_action['ACTION'] = lp_action['ACTION'].replace({'INCREASE_LIQUIDITY':'Increase Liquidity',
                                                   'DECREASE_LIQUIDITY':'Decrease Liquidity'})
lp_action.rename(columns = {'LIQUIDITY_PROVIDER':'SENDER', 'ACTION':'EVENT_NAME'}, inplace = True)

# Get lower and upper price from tick
lp_action['PRICE_LOWER'] = lp_action['TICK_UPPER'].apply(Get_Price)
lp_action['PRICE_UPPER'] = lp_action['TICK_LOWER'].apply(Get_Price)

# Reverse to make sure LPs are sorted in the same order as they are executed
lp_action = lp_action.iloc[::-1]
lp_action.reset_index(drop = True, inplace = True)

lp_action.head()

,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 500 10,Increase Liquidity,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,977,2995.507735,USDC,2995.292327,1.000000,WETH,3437.435736,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
1,USDC-WETH 500 10,Decrease Liquidity,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,977,2588.187276,USDC,2588.001158,0.931484,WETH,3201.917248,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
2,USDC-WETH 500 10,Increase Liquidity,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,1242,1.000000,USDC,0.999928,0.000146,WETH,0.500964,194990,195010,3398.18281290132699723471887409687042236328125,3404.98563895004917867481708526611328125
3,USDC-WETH 500 10,Increase Liquidity,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1428,598.469729,USDC,598.426693,2.600000,WETH,8937.332900,186730,195460,3248.661520935142107191495597362518310546875,7777.2237085991100684623233973979949951171875
4,USDC-WETH 500 10,Increase Liquidity,12377308,2021-05-05 23:44:49,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1464,3405.610348,USDC,3405.365449,0.000000,WETH,0.000000,194660,194990,3404.98563895004917867481708526611328125,3519.21893691676132220891304314136505126953125


# Fill missing NF_Token_ID

In [3]:
missing_NF_Token_ID = lp_action.loc[lp_action['NF_TOKEN_ID'] == 'nan', ['TICK_LOWER','TICK_UPPER']].drop_duplicates().reset_index(drop = True)
missing_NF_Token_ID = missing_NF_Token_ID.rename_axis('self_NF_TOKEN_ID').reset_index()
missing_NF_Token_ID['self_NF_TOKEN_ID'] = ['ID_' + str(x) for x in missing_NF_Token_ID['self_NF_TOKEN_ID']]

lp_action = lp_action.merge(missing_NF_Token_ID, how = 'left', on = ['TICK_LOWER','TICK_UPPER'])
lp_action['NF_TOKEN_ID'] = np.where(lp_action['NF_TOKEN_ID'] == 'nan', lp_action['self_NF_TOKEN_ID'], lp_action['NF_TOKEN_ID'])
lp_action.drop(columns = ['self_NF_TOKEN_ID'], inplace = True)
lp_action.head()

,POOL_NAME,EVENT_NAME,BLOCK_NUMBER,BLOCK_TIMESTAMP,SENDER,NF_TOKEN_ID,AMOUNT0_ADJUSTED,TOKEN0_SYMBOL,AMOUNT0_USD,AMOUNT1_ADJUSTED,TOKEN1_SYMBOL,AMOUNT1_USD,TICK_LOWER,TICK_UPPER,PRICE_LOWER,PRICE_UPPER
0,USDC-WETH 500 10,Increase Liquidity,12376729,2021-05-05 21:42:11,0xb2ef52180d1e5f4835f4e343251286fa84743456,977,2995.507735,USDC,2995.292327,1.000000,WETH,3437.435736,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
1,USDC-WETH 500 10,Decrease Liquidity,12376958,2021-05-05 22:27:41,0xb2ef52180d1e5f4835f4e343251286fa84743456,977,2588.187276,USDC,2588.001158,0.931484,WETH,3201.917248,191150,198080,2499.91359912486268513021059334278106689453125,4998.9181710649063461460173130035400390625
2,USDC-WETH 500 10,Increase Liquidity,12377035,2021-05-05 22:44:11,0x35f5a1ec10deee1256351e37bc78dc80345895f8,1242,1.000000,USDC,0.999928,0.000146,WETH,0.500964,194990,195010,3398.18281290132699723471887409687042236328125,3404.98563895004917867481708526611328125
3,USDC-WETH 500 10,Increase Liquidity,12377266,2021-05-05 23:34:39,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1428,598.469729,USDC,598.426693,2.600000,WETH,8937.332900,186730,195460,3248.661520935142107191495597362518310546875,7777.2237085991100684623233973979949951171875
4,USDC-WETH 500 10,Increase Liquidity,12377308,2021-05-05 23:44:49,0x9f9b987becfe15aca93ce9208cca146f252c8d46,1464,3405.610348,USDC,3405.365449,0.000000,WETH,0.000000,194660,194990,3404.98563895004917867481708526611328125,3519.21893691676132220891304314136505126953125


# Add Liquidity Parameter

In [7]:
with mp.Pool(10) as pool:
    result = pool.imap(Get_Liquidity_Parameter, lp_action[['AMOUNT1_ADJUSTED','AMOUNT0_ADJUSTED','PRICE_LOWER','PRICE_UPPER']].iterrows(), chunksize = 1)
    output = [x for x in result]

lp_action['L'] = output

Working on row: 0
Working on row: 10000
Working on row: 20000
Working on row: 30000
Working on row: 40000
Working on row: 50000
Working on row: 60000
Working on row: 70000
Working on row: 80000
Working on row: 90000
Working on row: 100000
Working on row: 110000
Working on row: 120000
Working on row: 130000
Working on row: 140000
Working on row: 150000
Working on row: 160000


In [8]:
lp_action.to_csv(os.path.join(data_dir, "uniswapv3_lp_actions", pool_name + "_LP.csv"), index=False)